In [ ]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/stratified-k-folds-tps-oct-21/train_folds.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/test.csv')
sample = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
import optuna
from xgboost import XGBClassifier

In [ ]:
features = [c for c in df.columns if c not in ('id','target','fold')]
from sklearn.metrics import roc_auc_score


# def objective(trial):
#     params = {
#         'n_estimators':10000,
#         'eval_metric': 'auc',
#         'learning_rate': trial.suggest_loguniform('learning_rate',1e-3,0.5),
#         'booster':'gbtree',
#         'use_label_encoder': False,
#         'lambda':trial.suggest_loguniform('lamba',1,100),
#         'alpha':trial.suggest_loguniform('alpha',0.5,100),
#         'subsample':trial.suggest_loguniform('subsample',0.2,1),
#         'colsample_bytree':trial.suggest_loguniform('colsample_bytree',0.1,1),
#         'max_depth':trial.suggest_int('max_depth',1,8),
#         'tree_method': 'gpu_hist',
#         'gpu_id': 0,
#         'random_state':42
#     }
    
#     fold = 1
    
    
#     xtrain = df[df.fold != fold].reset_index(drop=True)
#     xvalid = df[df.fold == fold].reset_index(drop=True)
    
#     ytrain = xtrain.target
#     yvalid = xvalid.target
    
#     xtrain = xtrain[features]
#     xvalid = xvalid[features]
    
    
    
#     model = XGBClassifier(**params)
#     model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    
#     preds = model.predict_proba(xvalid)[:,-1]
    
#     auc = roc_auc_score(yvalid, preds)
#     print(f"xgb auc: {auc:.6f}")
#     return auc

In [ ]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100)

In [ ]:
# study_params = study.best_params

# study_params

In [ ]:
# from optuna.visualization import plot_optimization_history
# from optuna.visualization import plot_param_importances

# plot_optimization_history(study)

In [ ]:
# plot_param_importances(study)

In [ ]:
params = {
    'n_estimators':10000,
    'eval_metric': 'auc',
    'booster':'gbtree',
    'use_label_encoder': False,
    'tree_method': 'gpu_hist',
    'gpu_id': 0,
    'random_state':42,
    'learning_rate': 0.009541394257779908,
    'lamba': 57.646976288208826,
    'alpha': 3.5229278217190783,
    'subsample': 0.5683776427424319,
    'colsample_bytree': 0.3294019517978491,
    'max_depth': 5
}
final_predictions = []
for fold in range(5):
    xtrain = df[df.fold != fold].reset_index(drop=True)
    xvalid = df[df.fold == fold].reset_index(drop=True)
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    xtest = test.copy()
    xtest = xtest[features]
    
    
    model = XGBClassifier(**params)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    
    preds_valid = model.predict_proba(xvalid)[:,-1]
    preds_test = model.predict_proba(xtest)[:,-1]
    final_predictions.append(preds_test)
    print(f"fold {fold} - xgb auc: {roc_auc_score(yvalid, preds_valid):.6f}")
    

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)
sample.target = preds
sample.to_csv("xgb_submission", index=False)